## Important Dependencies with their required versions


In [ ]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb
# !pip install text_generation

## Imports and Setup and Initializing Agent

In [1]:
# # to auto reload instead of having to restart each time
%load_ext autoreload
%autoreload 2

In [ ]:
# from langchain_community.llms import HuggingFaceEndpoint, HuggingFaceHub
# import os
# from dotenv import load_dotenv
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate

# question = "Who won the FIFA World Cup in the year 1994? "

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = PromptTemplate.from_template(template)

# repo_id = "bigcode/starcoder2-7b"
# load_dotenv()
# # llm = HuggingFaceEndpoint(
# #     repo_id=repo_id, max_new_tokens=12000, temperature=0.5, token= os.environ.get("HUGGINGFACE_TOKEN")
# # )

# llm = HuggingFaceHub(
#         repo_id=repo_id,
#         huggingfacehub_api_token= os.environ.get("HUGGINGFACE_TOKEN"),
#         task="text-generation",
#         model_kwargs={
#             "temperature": 0.1,  # can't be 0
#             "max_new_tokens": 12000,
#         },
#         cache=False,
#     )

# llm_chain = LLMChain(prompt=prompt, llm=llm)
# print(llm_chain.run({"question":question}))

In [1]:
# from Imports import *
try:
    from Configuration import *
    from TestGenerator import *
    from TestFix import *

    llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"])

    t = TestGenerator(GenUnitTestChain, db, HEval_JsonObj, globals())
    f = TestFix(globals(), chat_model)
except Exception as e:
    print(e)
    exit(-1)

C:\Users\omare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
C:\Users\omare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you in

## Generate Unit Tests from Code

In [ ]:
# llm("What is 4 + 5?")

In [2]:
t.generate()

NameError: name 't' is not defined

In [2]:
f.generate()

Running Example  0 

Example 0  has already passed
Running Example  1 

<s>[INST] 
you generated unittest for the following method under test having the following description:
*Method under test*:

from typing import List


def mean_absolute_deviation(numbers: List[float]) -> float:
    mean = sum(numbers) / len(numbers)
    return sum(abs(x - mean) for x in numbers) / len(numbers)


*Description*:
    For a given list of input numbers, calculate Mean Absolute Deviation
    around the mean of this dataset.
    Mean Absolute Deviation is the average absolute difference between each
    element and a centerpoint (mean in this case):
    MAD = average | x - x_mean |
    >>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
    1.0


but this test failed
*Test*:
    def test_empty_list(self):
        self.assertIsNone(mean_absolute_deviation([]))


    def test_single_element(self):

with the following feedback:
*Feedback*:
ERROR: test_empty_list (__main__.TestMeanAbsoluteDeviation)
Traceback 